In [2]:
# using Pkg
# Pkg.add("DFTK")

using ParallelTemperingMonteCarlo
using DelimitedFiles


    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


   Installed EzXML ───────────── v1.1.0
   Installed OffsetArrays ────── v1.12.8


   Installed LLVMOpenMP_jll ──── v15.0.4+0
   Installed MKL_jll ─────────── v2022.2.0+0
   Installed TimerOutputs ────── v0.5.22
   Installed StructHelpers ───── v0.1.5
   Installed Roots ───────────── v2.0.8
   Installed MPI ─────────────── v0.20.4
   Installed DFTK ────────────── v0.5.14


   Installed MPItrampoline_jll ─ v5.0.2+1


...

  ✓ FFTW


  ✓ InteratomicPotentials


  ✓ DFTK


  ✓ ParallelTemperingMonteCarlo


  24 dependencies successfully precompiled in 79 seconds. 89 already precompiled.


In [3]:
using Random#,Plots

#set random seed - for reproducibility
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 4.
tf = 19.
n_traj = 22

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 100000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=6.2*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)



Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

In [6]:
ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false)
#plot(temp.t_grid,results.heat_cap)

data = [results.en_histogram[i] for i in 1:n_traj]
plot(data)

Total number of moves per MC cycle: 13



MethodError: MethodError: no method matching energy_update(::StaticArraysCore.SVector{3, Float64}, ::Int64, ::Config{13, SphericalBC{Float64}, Float64}, ::Matrix{Float64}, ::ELJPotentialEven{6, Float64})
Closest candidates are:
  energy_update(::Any, ::Any, ::Config, ::Any, ::Any, !Matched::DFTPotential) at ~/ParallelTemperingMonteCarlo.jl/src/EnergyEvaluation.jl:246
  energy_update(::Any, ::Any, ::Any, ::Any, ::Any, !Matched::AbstractDimerPotential) at ~/ParallelTemperingMonteCarlo.jl/src/EnergyEvaluation.jl:103
  energy_update(::Any, ::Any, ::Any, ::Any, !Matched::AbstractMLPotential) at ~/ParallelTemperingMonteCarlo.jl/src/EnergyEvaluation.jl:110
  ...

In [5]:
pptmc_run!(mc_states,move_strat,mc_params,pot,ensemble,results)


Beginning Equilibration




TaskFailedException: TaskFailedException

    nested task error: MethodError: no method matching energy_update(::StaticArraysCore.SVector{3, Float64}, ::Int64, ::Config{13, SphericalBC{Float64}, Float64}, ::Matrix{Float64}, ::ELJPotentialEven{6, Float64})
    Closest candidates are:
      energy_update(::Any, ::Any, ::Config, ::Any, ::Any, !Matched::DFTPotential) at ~/ParallelTemperingMonteCarlo.jl/src/EnergyEvaluation.jl:246
      energy_update(::Any, ::Any, ::Any, ::Any, ::Any, !Matched::AbstractDimerPotential) at ~/ParallelTemperingMonteCarlo.jl/src/EnergyEvaluation.jl:103
      energy_update(::Any, ::Any, ::Any, ::Any, !Matched::AbstractMLPotential) at ~/ParallelTemperingMonteCarlo.jl/src/EnergyEvaluation.jl:110
      ...
    Stacktrace:
     [1] atom_move!(mc_state::MCState{Float64, 13, SphericalBC{Float64}}, i_atom::Int64, pot::ELJPotentialEven{6, Float64}, ensemble::NVT)
       @ ParallelTemperingMonteCarlo.MCRun ~/ParallelTemperingMonteCarlo.jl/src/MCRun.jl:240
     [2] mc_step